In [2]:
from pyspark.sql import SparkSession
Spark=SparkSession.builder.master("local[*]").appName('Pattern Recognizer Model').getOrCreate()

In [3]:
#data=spark.read.csv('file:///home/chiranjib/spark_works/Pattern_Recognition/dataset/iris.csv',header=True,inferSchema=True)
data=spark.read.csv('file:///home/chiranjib/spark_works/Pattern_Recognition/dataset/wine_modified.csv',header=True,inferSchema=True)
#data=spark.read.csv('file:///home/chiranjib/spark_works/Pattern_Recognition/dataset/pima-indians-diabetes_modified.csv',header=True,inferSchema=True)

In [4]:
data.columns

['Class',
 'Alcohol',
 'Malic_acid',
 'Ash',
 'Alcalinity_of_ash',
 'Magnesium',
 'Total_phenols',
 'Flavanoids',
 'Nonflavanoid_phenols',
 'Proanthocyanins',
 'Color_intensity',
 'Hue',
 'OD_of_diluted_wines',
 'Proline']

In [5]:
data.show(5)

+-----+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+-------------------+-------+
|Class|Alcohol|Malic_acid| Ash|Alcalinity_of_ash|Magnesium|Total_phenols|Flavanoids|Nonflavanoid_phenols|Proanthocyanins|Color_intensity| Hue|OD_of_diluted_wines|Proline|
+-----+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+-------------------+-------+
|    1|  14.23|      1.71|2.43|             15.6|      127|          2.8|      3.06|                0.28|           2.29|           5.64|1.04|               3.92|   1065|
|    1|   13.2|      1.78|2.14|             11.2|      100|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|                3.4|   1050|
|    1|  13.16|      2.36|2.67|             18.6|      101|          2.8|      3.24|                 0.3|           2.81|           5.68|1.03|   

In [6]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import *

In [7]:
# for iris dataset........
# update_column=(when(data.variety == "Setosa",0) \
#       .when(data.variety == "Versicolor",1) \
#       .when(data.variety == "Virginica",2))

# updated_data=data.withColumn("variety",update_column)
# assembler=VectorAssembler(inputCols=updated_data.columns,outputCol='features')
# assembled=assembler.transform(updated_data)


# for wine dataset........
update_column=(when(data.Class == 1,0) \
      .when(data.Class == 2,1) \
      .when(data.Class == 3,2))

updated_data=data.withColumn("Class",update_column)
assembler=VectorAssembler(inputCols=updated_data.columns,outputCol='features')
assembled=assembler.transform(updated_data)


# for pima dataset........
# assembler=VectorAssembler(inputCols=data.columns,outputCol='features')
# assembled=assembler.transform(data)

In [8]:
(train_set,test_set)=assembled.randomSplit([0.5,0.5])
train_set.count(),test_set.count()

(88, 90)

In [11]:
labeled_set,unlabeled_pool_set=train_set.randomSplit([0.1,0.9])
labeled_set.count(),unlabeled_pool_set.count()

(10, 78)

In [12]:
#NB=NaiveBayes(featuresCol='features',labelCol='variety',modelType='multinomial')
NB=NaiveBayes(featuresCol='features',labelCol='Class',modelType='multinomial')
i=1
while True:
    print("Iteration",i,"...")
    #pipeline=Pipeline(stages=[NB])
    model=NB.fit(labeled_set)
    predictions=model.transform(test_set)
    #evaluator=MulticlassClassificationEvaluator(labelCol='variety',predictionCol='prediction')
    evaluator=MulticlassClassificationEvaluator(labelCol='Class',predictionCol='prediction')
    accuracy=evaluator.evaluate(predictions)
    n=unlabeled_pool_set.count()
    print("Accuracy :",accuracy*100,"unlabeled_set=",n)
    if accuracy*100<90 and n>0 :
        addition=unlabeled_pool_set.sample(False,0.10)
        labeled_set=labeled_set.union(addition)
        unlabeled_pool_set=unlabeled_pool_set.subtract(addition)
    else:
        break
    i+=1
print("Done!!")

Iteration 1 ...
Accuracy : 79.16159000913798 unlabeled_set= 78
Iteration 2 ...
Accuracy : 71.03648611424136 unlabeled_set= 71
Iteration 3 ...
Accuracy : 82.11128640472792 unlabeled_set= 64
Iteration 4 ...
Accuracy : 92.25026980725404 unlabeled_set= 58
Done!!
